In [1]:
#Ejemplo en Notebook sobre como se ejecuta cada bloque en compileNFT.py de la rama main 
# Nota: ejecutamos sobre una blockchain privada mediante Ganache, conviene tenerse instalado con 
# npm install ganache --global

#Importamos las librerías
from web3 import Web3
import solcx
from solcx import compile_source
import json
import os

# Nota: dependiendo del compilador que necesite tu contrato es recomendable instalar la versión correspondiente 
solcx.install_solc('0.8.1')



Version('0.8.1')

In [3]:
# Nos conectamos al RPC de la blockchain dada, en este caso la local de Ganache
web3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))


In [30]:
# obtenemos el contrato ya sea mediante Jinja2, hardcodeado en strings o abriendolo con os 
contract_source_code = '''
pragma solidity ^0.8.0;

import "node_modules/@openzeppelin/contracts/token/ERC721/ERC721.sol";

contract GB_NFT is ERC721 {
    string private _name;
    string private _symbol;

    constructor(string memory name, string memory symbol) ERC721(name, symbol) {
        _name = name;
        _symbol = symbol;
    }

    function mint(address to, uint256 tokenId) public {
        _mint(to, tokenId);
    }

    function updateNameAndSymbol(string memory name, string memory symbol) public {
        require(_isApprovedOrOwner(msg.sender, 1), "Only the owner can update the name and symbol");
        _name = name;
        _symbol = symbol;
    }

    function name() public view virtual override returns (string memory) {
        return _name;
    }

    function symbol() public view virtual override returns (string memory) {
        return _symbol;
    }
}

'''

In [31]:
# Compilamos el contrato y obtenmos la interfaz del contrato, de donde obtenemos el ABI
compiled_sol = compile_source(contract_source_code)
contract_interface = compiled_sol['<stdin>:GB_NFT']

In [32]:
# Desplegamos el contrato con los parámetros generados
contract = web3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

#Nota: aquí el nombre del NFT y el símbolo está hardcodeado pero se puede implementar desde otro lado dentro del py
tx_hash = contract.constructor('MyNFTminting', 'MNFTm').transact({'from': '0x85C5AbD48fe34258af20451f90934e7E0c39D9C8'})
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

#Address dueña del contrato
contract_address = tx_receipt.contractAddress

In [33]:
#Generamos instancia de contrato generado con la libreria web3.py 
contract_instance = web3.eth.contract(address=contract_address, abi=contract_interface['abi'])

In [59]:
# Con este comando podemos enlistar todas las funciones disponibles en el contrato
contract_functions = contract_instance.all_functions()
self_address = '0x85C5AbD48fe34258af20451f90934e7E0c39D9C8'

# Ejemplo: Mintear un NFT del contrato con un ID dado a
# Mint hacia self_address con TokenID dado
tx_mint = contract_instance.functions.mint(self_address, 1).transact({'from': self_address})

# para comprobar si el NFT existe y cual es la wallet donde está llamamos a la funcion ownerOf del contrato 
owner = contract_instance.functions.ownerOf(1).call()

In [75]:
# Y efectivamente, owner = self_address
print(owner)

In [77]:

# Ejemplo #2: cambiar nombre al contrato de NFT a otro que se decida
contract_instance.functions.name().call()

# Vemos que por el constructor del contrato los NFTs se llaman "MNFTm" y la colección es "MyNFTminting"


'GBNFT nuevo'

In [ ]:
# Llamamos a la funcion updateNameAndSymbol y cambiamos el nombre y símbolo
# Nota: el nombre y símbolo están hardcodeados, se pueden implementar en el mismo py  
tx_hash = contract_instance.functions.updateNameAndSymbol('GBNFT nuevo', 'GBNFT').transact({'from':self_address})

In [ ]:
# Vemos que no son los mismos
contract_instance.functions.name().call()
contract_instance.functions.symbol().call()